#  Project Title : Sentiment Analysis of Twitter Tweets of US Airlines

# Loading up the dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Loading the Test and Train Data

In [2]:
train = pd.read_csv("/Users/arunavdey/iCloud Drive (Archive)/Desktop/Archive/TRANSFER/Machine Learning Course Data/twitter_train.csv")
test = pd.read_csv("/Users/arunavdey/iCloud Drive (Archive)/Desktop/Archive/TRANSFER/Machine Learning Course Data/twitter_test.csv")

#  Preview of the Training Data

In [3]:
train.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [4]:
test.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


# Count of items of each feature 

In [5]:
train.count()

tweet_id                  10980
airline_sentiment         10980
airline                   10980
airline_sentiment_gold       31
name                      10980
negativereason_gold          24
retweet_count             10980
text                      10980
tweet_coord                 776
tweet_created             10980
tweet_location             7430
user_timezone              7403
dtype: int64

#  Dropping the irrelevant features

In [6]:
# Dropping Useless Columns

train.drop('tweet_id', inplace = True, axis = 1)
train.drop('airline_sentiment_gold', inplace = True, axis = 1)
train.drop('negativereason_gold', inplace = True, axis = 1)
train.drop('tweet_coord', inplace = True, axis = 1)
train.drop('tweet_created',inplace = True, axis = 1)

# Preview of the modified Train data 

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  10980 non-null  object
 1   airline            10980 non-null  object
 2   name               10980 non-null  object
 3   retweet_count      10980 non-null  int64 
 4   text               10980 non-null  object
 5   tweet_location     7430 non-null   object
 6   user_timezone      7403 non-null   object
dtypes: int64(1), object(6)
memory usage: 600.6+ KB


In [8]:
train.head()

,airline_sentiment,airline,name,retweet_count,text,tweet_location,user_timezone
0,negative,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",Washington D.C.,Atlantic Time (Canada)
1,positive,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,positive,United,LocalKyle,0,@united Flew ORD to Miami and back and had gr...,Illinois,Central Time (US & Canada)
3,negative,Southwest,amccarthy19,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,Atlantic Time (Canada)
4,negative,United,J_Okayy,0,@united so our flight into ORD was delayed bec...,NaN,Eastern Time (US & Canada)


# Finding NaN values (if any)

In [9]:
mood_count = train['airline_sentiment'].value_counts()
mood_count

negative    6851
neutral     2327
positive    1802
Name: airline_sentiment, dtype: int64

In [10]:
train['airline'].value_counts()

United            2928
US Airways        2152
American          2078
Southwest         1817
Delta             1639
Virgin America     366
Name: airline, dtype: int64

# Assigning labels to the classes 'negative','positive','neutral' by using LabelEncoder

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['airline_sentiment'] = le.fit_transform(train['airline_sentiment'])
train.head()

,airline_sentiment,airline,name,retweet_count,text,tweet_location,user_timezone
0,0,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",Washington D.C.,Atlantic Time (Canada)
1,2,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,2,United,LocalKyle,0,@united Flew ORD to Miami and back and had gr...,Illinois,Central Time (US & Canada)
3,0,Southwest,amccarthy19,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,Atlantic Time (Canada)
4,0,United,J_Okayy,0,@united so our flight into ORD was delayed bec...,NaN,Eastern Time (US & Canada)


# Importing NLTK

In [12]:
import re
import nltk
from nltk.corpus import stopwords

# Writing a custom function to perform the following  functions on each tweet text:
#### 1.Lowering the case of each letter in each and every tweet
#### 2.Removing the stop words , other non required symbols and annotations 

In [13]:
def tweet_to_words(raw_tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return( " ".join( meaningful_words )) 

# Adding new column 'cleaned_tweet' which contains the 'cleaned' form of the tweet

In [14]:
train['cleaned_tweet'] = train['text'].apply(lambda x: tweet_to_words(x))

test['cleaned_tweet'] = test['text'].apply(lambda x: tweet_to_words(x))

# Preview of the data

In [15]:
train.head()

,airline_sentiment,airline,name,retweet_count,text,tweet_location,user_timezone,cleaned_tweet
0,0,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",Washington D.C.,Atlantic Time (Canada),southwestair scheduled morning days fact yes s...
1,2,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,"Indianapolis, Indiana; USA",Central Time (US & Canada),southwestair seeing workers time time going be...
2,2,United,LocalKyle,0,@united Flew ORD to Miami and back and had gr...,Illinois,Central Time (US & Canada),united flew ord miami back great crew service ...
3,0,Southwest,amccarthy19,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,Atlantic Time (Canada),southwestair dultch horse radish
4,0,United,J_Okayy,0,@united so our flight into ORD was delayed bec...,NaN,Eastern Time (US & Canada),united flight ord delayed air force one last f...


In [16]:
test.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,cleaned_tweet
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada),americanair car gng dfw pulled hr ago icy road...
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada),americanair plane land identical worse conditi...
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada),southwestair believe many paying customers lef...
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada),usairways legitimately say would rather driven...
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada),americanair still response aa great job guys


# Appending each value of the column 'cleaned_tweet' in respective test-train lists

In [17]:
train_clean_tweet=[]
for tweet in train['cleaned_tweet']:
    train_clean_tweet.append(tweet)
    
test_clean_tweet=[]
for tweet in test['cleaned_tweet']:
    test_clean_tweet.append(tweet)

# Count Vectorization of the Data

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(analyzer = "word")
train_features= v.fit_transform(train_clean_tweet)
test_features= v.transform(test_clean_tweet)

# Preview of the Vectorized Data

In [19]:
train_features

<10980x11471 sparse matrix of type '<class 'numpy.int64'>'
	with 107004 stored elements in Compressed Sparse Row format>

In [20]:
test_features

<3660x11471 sparse matrix of type '<class 'numpy.int64'>'
	with 33563 stored elements in Compressed Sparse Row format>

# Importing sklearn Naive Bayes

In [21]:
from sklearn.naive_bayes import MultinomialNB

In [22]:
nb = MultinomialNB()

# Implementation of Naive Bayes on the Data

In [23]:
nb.fit(train_features,train['airline_sentiment'])
from sklearn.metrics import accuracy_score, confusion_matrix

# Decoding the values of the column 'airline_sentiment' back to the 'negative-positive' form

In [24]:
test_pred_nb = nb.predict(test_features)
test_pred_nb = le.inverse_transform(test_pred_nb)

# Preview of the predicted values 

In [25]:
print(test_pred_nb)

['negative' 'negative' 'negative' ... 'neutral' 'positive' 'neutral']


# Accuracy of the Algorithm on the Training Data

In [26]:
accuracy_score(nb.predict(train_features),train['airline_sentiment'])

0.8437158469945355

# Now trying some other Classifier for doing predictions

# Importing sklearn Random Forests

In [27]:
from sklearn.ensemble import RandomForestClassifier

# Classification using Random Forest Classifier

In [28]:
clf = RandomForestClassifier(n_estimators = 200)

In [29]:
clf.fit(train_features,train['airline_sentiment'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# Accuracy of the algorithm in Training Data

In [30]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy = accuracy_score(clf.predict(train_features),train['airline_sentiment'])
print(accuracy)

0.995719489981785


# Predictions using Random Forest 

In [31]:
test_pred_rf=clf.predict(test_features)

# Decoding the values of the column 'airline_sentiment' back to the 'negative-positive' form

In [32]:
test_pred_rf=le.inverse_transform(test_pred_rf)

# Preview of the predictions

In [33]:
test_pred_rf

array(['negative', 'negative', 'negative', ..., 'neutral', 'positive',
       'negative'], dtype=object)

# Saving the Predicted values from the Random Forests Classifier into CSV file.

In [35]:
np.savetxt("/Users/arunavdey/iCloud Drive (Archive)/Desktop/Archive/TRANSFER/Machine Learning Course Data/twitter_pred_rf.csv",test_pred_rf,fmt='%s')

# RESULT:
### Hence, we conducted the Sentiment Analysis of Twitter Tweets of US Airlines,we noticed that we were able to get an Accuracy Score of 0.995 by the use of Random Forests Classifier.